In [ ]:
import numpy as np
from pydrake.all import (
    RigidTransform,
    RotationMatrix,
    StartMeshcat,
    RandomGenerator,
    Diagram,
)
import numpy as np

import sponana.utils
from sponana.planner.rrt import SpotProblem, rrt_planning
from sponana.planner.navigator import Navigator
from sponana.fsm.finite_state_machine import FiniteStateMachine
import sponana.sim

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
meshcat.Delete()
meshcat.DeleteAddedControls()
rng = np.random.default_rng(145)  # this is for python
generator = RandomGenerator(rng.integers(0, 1000))  # this is for c++

add_spot = True
# simulation_time = -1  # run indefinitely until ESC is pressed
simulation_time = 10
debug = True
add_fixed_cameras = False
enable_arm_ik = True  # turn this off if you find the arm too annoying
use_teleop = False

simulator, diagram = sponana.sim.clutter_gen(
    meshcat,
    rng,
    debug=debug,
    simulation_time=simulation_time,
    add_spot=add_spot,
    add_fixed_cameras=add_fixed_cameras,
    enable_arm_ik=enable_arm_ik,
    use_teleop=use_teleop,
)

In [16]:
initial_pose = np.array([1.00000000e00, 1.50392176e-12, 3.15001955e00])
q_start = initial_pose
camera_pose0 = np.array([1.00000000e00, 1.50392176e-12, 3.15001955e00])
camera_pose1 = np.array([0.20894849, -0.47792893, 0.2475])
camera_pose2 = np.array([0.20894849, -0.47792893, 0.2475])


SyntaxError: invalid syntax (2633313655.py, line 12)

In [ ]:
def update_do_rrt(current_cam_reached, current_cam_ind, check_banana):
        """
        Function to update flag to indicate to RRT/Navigator for ready for planning/movement.
        Inputs: 
        - from context, 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        
        Returns: 
        - new_cam_reached: if current cam reached, switch to 0 so that navigator can plan again. 
        if current cam is not reached, stay 1 so that navigator will wait. 
        """
        do_rrt = 0
        #just starting, have not reached the first camera pose, do_rrt to get to the first camera
        if current_cam_reached == 0 and current_cam_ind == 1:
            do_rrt = 1
        elif current_cam_reached == 1 and check_banana == 1: 
            do_rrt = 1
        
        return do_rrt

In [ ]:
def update_camera_ind(current_cam_reached, see_banana, has_banana, current_cam_ind):
        """Function for updating camera pose list index. 
        Inputs: 
        - from context: 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        see_banana: int where 0 when banana is not seen and 1 when banana seen. 
        has_banana: int where 0 when banana is grasped nad 1 when banana is not grasped. 

        Returns: None
        If current camera is reached and no banana is seen/grasped, needs to continue to search
        next camera pose, so current_cam_ind is incremented. 
        """ 
        if current_cam_reached == 1 and see_banana == 0 and has_banana == 0:
            if current_cam_ind <= 7:
                current_cam_ind += 1
            #none viewpoints have bananas, so do it all again?
            else:
                current_cam_ind = 0

        return current_cam_ind


In [ ]:
def get_camera_pose(current_cam_ind, camera_pose_list):
        """
        Function to get the next camera pose for Spot to travel to in RRT
        Inputs: 
        - from context: 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        Returns: 
        - next camera pose
        """
        next_camera_pose = camera_pose_list[current_cam_ind]
        return next_camera_pose

In [ ]:
def update_check_banana(current_cam_reached, see_banana, has_banana):
        """Function as indicater for perception module.
        Inputs: 
        - from context: 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        see_banana: int where 0 when banana is not seen and 1 when banana seen. 
        has_banana: int where 0 when banana is grasped nad 1 when banana is not grasped. 

        Returns: 
        check_banana: if current_cam is reached, and banana is not seen, and banana is not grasped, 
        return 1 to call the perception module/system. 
        """
        check_banana = 0
        if current_cam_reached == 1 and see_banana == 0 and has_banana == 0:
            check_banana = 1
        return check_banana

In [ ]:
def update_grasp_banana(current_cam_reached, see_banana, has_banana):
        """Function as indicater for perception module.
        Inputs: 
        - from context: 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        see_banana: int where 0 when banana is not seen and 1 when banana seen. 
        has_banana: int where 0 when banana is grasped nad 1 when banana is not grasped. 

        Returns: 
        grasp_banana: if current_cam is reached, and banana is seen, and banana is not grasped, 
        return 1 to get banana grasped system. 
        """
        grasp_banana = 0
        if current_cam_reached == 1 and see_banana == 1 and has_banana == 0:
            grasp_banana = 1
        return grasp_banana

In [ ]:
def update_completion(has_banana):
    completed = 0
    if has_banana == 1:
        completed = 1
    return completed

In [ ]:
def execute_finite_state_machine():
    while complete_flag == 0:
        check_do_rrt = self._update_do_rrt(context)
        self._update_camera_ind(context, state)
        next_camera_pose = self._get_camera_pose(context)
        check_banana = self._update_check_banana(context)
        grasp_banana = self._update_grasp_banana(context)
        completed = self._update_completion(context)

In [ ]:
navigator = Navigator()
spot_problem = SpotProblem(
    q_start=q_start, q_goal=q_goal, collision_checker=navigator._collision_check
)
path = rrt_planning(spot_problem, 1000, 0.05)

In [ ]:
# visualize RRT output
from manipulation.meshcat_utils import AddMeshcatTriad


def visualize_path(path):
    for i, pose in enumerate(path):
        pose = RigidTransform(RotationMatrix.MakeZRotation(pose[2]), [*pose[:2], 0.0])
        opacity = 0.2
        AddMeshcatTriad(meshcat, f"trajectory_{i}", X_PT=pose, opacity=opacity)


visualize_path(path)

In [ ]:
sponana.utils.visualize_diagram(diagram, 1)